In [ ]:
from dataclasses import dataclass
from typing import Any, Sequence


@dataclass
class Backend:
    HLayout: Any
    VLayout: Any
    Label: Any
    Button: Any
    ToggleButton: Any
    MainWindow: Any
    Slider: Any
    ComboBox: Any
    SpinBox: Any
    ScrollArea: Any
    GroupBox: Any
    HtmlView: Any


def make_qt_backend() -> Backend:
    from PySide6.QtCore import Qt
    from PySide6.QtGui import QFont
    from PySide6.QtWidgets import (
        QApplication,
        QComboBox,
        QDoubleSpinBox,
        QFrame,
        QHBoxLayout,
        QLabel,
        QPushButton,
        QScrollArea,
        QSlider,
        QVBoxLayout,
        QWidget,
        QGroupBox,
    )

    app = QApplication.instance()
    if app is None:
        app = QApplication([])

    class Slider(QSlider):
        MAX: int = 100_000_000

        def __init__(self, vmin: float, vmax: float, value: float, on_change: callable):
            super().__init__(Qt.Orientation.Horizontal)
            super().setMinimum(0)
            super().setMaximum(self.MAX)
            super().setValue(value)
            self._vmin = vmin
            self._vdel = vmax - vmin
            super().valueChanged.connect(lambda value: on_change(self._to_float(value)))

        def set_value(self, value: float):
            super().setValue(self._to_int(value))

        def _to_float(self, value):
            return self._vmin + (value / self.MAX) * (self._vdel)

        def _to_int(self, value):
            return int((value - self._vmin) / self._vdel * self.MAX)

    class ComboBox(QComboBox):
        def __init__(self, choices: Sequence[str], value: str, on_change: callable):
            super().__init__()
            for c in choices:
                super().addItem(c)
            super().setCurrentText(value)
            super().currentTextChanged.connect(on_change)

        def text(self):
            return super().currentText()

    class SpinBox(QDoubleSpinBox):
        def __init__(
            self,
            decimals: int,
            step: float,
            value: float,
            vmin: float,
            vmax: float,
            on_change: callable,
        ):
            super().__init__(
                value=value,
                decimals=decimals,
                singleStep=step,
                minimum=vmin,
                maximum=vmax,
            )
            super().valueChanged.connect(on_change)

        def set_value(self, value):
            super().setValue(value)

        def set_min(self, value):
            super().setMinimum(value)

        def set_max(self, value):
            super().setMaximum(value)

    class Button(QPushButton):
        def __init__(self, label, on_click):
            super().__init__(label)
            super().clicked.connect(on_click)

    class ToggleButton(QPushButton):
        def __init__(self, label, on_click, checked=False):
            super().__init__(label)
            super().setCheckable(True)
            super().setChecked(checked)
            super().clicked.connect(lambda: on_click(self.isChecked()))

        def set_checked(self, checked):
            super().setChecked(checked)

    class VLayout(QVBoxLayout):
        def __init__(self, *args):
            super().__init__()
            for arg in args:
                if isinstance(arg, (HLayout, VLayout)):
                    super().addLayout(arg)
                else:
                    super().addWidget(arg)

    class HLayout(QHBoxLayout):
        def __init__(self, *args):
            super().__init__()
            for arg in args:
                if isinstance(arg, (HLayout, VLayout)):
                    super().addLayout(arg)
                else:
                    super().addWidget(arg)

    class MainWindow(QWidget):
        def __init__(self, layout):
            super().__init__()
            font = QFont()
            font.setPointSize(11)
            self.setFont(font)
            self.setWindowTitle("iminuit")
            super().setLayout(layout)

        def render(self):
            super().show()
            app.exec()

    class Label(QLabel):
        def __init__(self, text, min_width: int = 0):
            super().__init__(text)
            if min_width > 0:
                super().setMinimumWidth(min_width)

        def set_text(self, text):
            super().setText(text)

    class ScrollArea(QScrollArea):
        def __init__(self, *args):
            super().__init__()
            super().setContentsMargins(0, 0, 0, 0)
            super().setFrameShape(QFrame.NoFrame)
            super().setWidgetResizable(True)
            child_widget = QWidget(self)
            super().setWidget(child_widget)
            # workaround for sizing issue
            super().setMinimumWidth(max(x.sizeHint().width() for x in args) + 30)

            layout = VLayout(*args)
            layout.setContentsMargins(0, 0, 0, 0)
            # if list is too short to fill full area,
            # add stretch at the end
            layout.addStretch()
            child_widget.setLayout(layout)

    class HtmlView(QLabel):
        def set_html(self, html: str):
            return super().setText(html)

    class GroupBox(QGroupBox):
        def __init__(self, layout, title: str = ""):
            super().__init__(title=title)
            super().setLayout(layout)

    return Backend(
        Button=Button,
        HLayout=HLayout,
        VLayout=VLayout,
        MainWindow=MainWindow,
        Label=Label,
        ToggleButton=ToggleButton,
        Slider=Slider,
        ComboBox=ComboBox,
        SpinBox=SpinBox,
        ScrollArea=ScrollArea,
        HtmlView=HtmlView,
        GroupBox=GroupBox,
    )


def make_ipy_backend() -> Backend:
    import ipywidgets as widgets
    from IPython.display import display

    class SpinBox(widgets.BoundedFloatText):
        def __init__(
            self,
            decimals: int,
            step: float,
            value: float,
            vmin: float,
            vmax: float,
            on_change: callable,
        ):
            super().__init__(value, min=vmin, max=vmax, step=step, decimals=decimals)
            super().observe(lambda event: on_change(self.value))

        def set_value(self, value):
            super().value = value

        def set_min(self, value):
            super().min = value

        def set_max(self, value):
            super().max = value

    class ComboBox(widgets.Dropdown):
        def __init__(self, choices: Sequence[str], value: str, on_change: callable):
            super().__init__(options=list(choices), value=value)
            super().observe(lambda event: on_change(self.value))

    class Slider(widgets.FloatSlider):
        def __init__(self, vmin: float, vmax: float, value: float, on_change: callable):
            super().__init__(
                min=vmin, max=vmax, value=value, continuous_update=True, readout=False
            )
            super().observe(lambda event: on_change(self.value))

        def set_value(self, value):
            self.value = value

    class Button(widgets.Button):
        def __init__(self, label, on_click):
            super().__init__(description=label)
            super().on_click(lambda *args: on_click())

    class HLayout(widgets.HBox):
        def __init__(self, *args):
            super().__init__(args)

    class VLayout(widgets.VBox):
        def __init__(self, *args):
            super().__init__(args)

    class Label(widgets.Label):
        def __init__(self, text, min_width=0):
            super().__init__(text)
            if min_width > 0:
                self.layout.min_width = f"{min_width}pt"

        def set_text(self, text):
            self.value = text

    class MainWindow:
        def __init__(self, layout):
            self.layout = layout

        def render(self):
            display(self.layout)

    class ToggleButton(widgets.ToggleButton):
        def __init__(self, label, on_click, checked=False):
            super().__init__(description=label, value=checked)
            super().observe(lambda event: on_click(self.value))

        def set_checked(self, checked):
            self.value = checked

    class ScrollArea(widgets.VBox):
        def __init__(self, *args):
            super().__init__(args)
            self.layout.overflow = "hidden scroll"
            self.layout.display = "flex"
            self.layout.flex_flow = "column"
            self.layout.max_height = "200px"
            self.layout.object_position = "center top"
            self.layout.align_items = "flex-start"

            # Make each child maintain its natural size
            for widget in args:
                widget.layout.min_height = "min-content"

    class HtmlView(widgets.HTML):
        def set_html(self, html: str):
            self.value = html

    class GroupBox(widgets.Box):
        def __init__(self, layout, title: str = ""):
            super().__init__(children=layout.children, layout=layout.layout)
            super().layout.border = "solid 1px"

    return Backend(
        Button=Button,
        HLayout=HLayout,
        VLayout=VLayout,
        MainWindow=MainWindow,
        Label=Label,
        ToggleButton=ToggleButton,
        Slider=Slider,
        ComboBox=ComboBox,
        SpinBox=SpinBox,
        ScrollArea=ScrollArea,
        HtmlView=HtmlView,
        GroupBox=GroupBox,
    )


# backend = make_ipy_backend()
backend = make_qt_backend()


def on_button1_click():
    print("Button 1 clicked!")


class MainWindow(backend.MainWindow):
    def __init__(self):
        self.b1 = backend.Button("Button1", on_button1_click)
        self.b2 = backend.Button("Button2", self.on_button2_click)
        self.label = backend.Label("Label", min_width=100)
        self.cb = backend.ToggleButton("Checker", self.on_checkbox)
        self.slider = backend.Slider(1, 4.5, 2.5, self.on_slider_change)
        self.counter = 0
        html_view = backend.HtmlView()
        layout = backend.HLayout(
            backend.VLayout(
                backend.HLayout(self.b1, self.b2),
                backend.HLayout(self.label, self.cb),
                self.slider,
                backend.ComboBox(("a", "bb", "ccc"), "bb", self.on_combobox_change),
                backend.SpinBox(
                    decimals=1,
                    step=0.1,
                    value=1.5,
                    vmin=0.5,
                    vmax=2.5,
                    on_change=self.on_spinbox_change,
                ),
                backend.GroupBox(
                    backend.HLayout(
                        backend.Button("Left", lambda: None),
                        backend.Button("Right", lambda: None),
                    )
                ),
                html_view,
            ),
            backend.ScrollArea(
                *[backend.Button(f"Button{i}", lambda: None) for i in range(20)]
            ),
        )
        html_view.set_html("<h1> Header </h1><p> Some text. <br> More text. </p>")

        super().__init__(layout)

    def on_button2_click(self):
        self.counter += 1
        self.label.set_text(f"counter={self.counter}")

    def on_checkbox(self, value):
        self.label.set_text(f"check={value}")

    def on_slider_change(self, value):
        self.label.set_text(f"slider={value:.1f}")

    def on_combobox_change(self, value):
        self.label.set_text(f"combo={value}")

    def on_spinbox_change(self, value):
        self.label.set_text(f"spin={value:.1f}")


mw = MainWindow()
mw.render()